In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

from nltk import RegexpParser
from nltk import pos_tag
import re
from fuzzywuzzy import process, fuzz

from keybert import KeyBERT

# doc = "WHOLE BLACK PEPPER LASTA (IMPORT TO EOU FOR RE- EXPORT)"
model = KeyBERT('distilbert-base-nli-mean-tokens')
# keywords = model.extract_keywords(doc)



def most_similar(doc_id,similarity_matrix,matrix,doc):
    #print (f'Document: {doc[doc_id]}')
    #print ('\n')
    #print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        #print('\n')
        #print (f'Document: {doc[ix]}')
        #print (f'{matrix} : {similarity_matrix[doc_id][ix]}')
    #print (f'{similarity_matrix[doc_id][-1]}')
    
    try:
        print(similarity_matrix[doc_id])
        return sorted(similarity_matrix[doc_id], reverse = True)[1]
    except:
        return 0


def key_ex(x):
    chunks = []
    for i in range(1,5):
        ls1 = model.extract_keywords(x, keyphrase_ngram_range=(1, i))

        for j in ls1:
            chunks.append(j)
    return chunks

def sel_top(val):
    final_val = []
    num = 0
    for value in val:
        print(value)
        if num > 5:
            break

        if value[1] > 0.3:
            final_val.append(value[0])
            num = num+1
    return final_val

# query = 'any MOTOR FOR INDUSTRIAL USE'

def semantic_check(query,df):
    result = []
    for x in df['Description'].values:
    #     doc = df['Description'].values
    #     doc.append(query)
#         print(x)
    #     chunks = []

        chunks = key_ex(x)
        chunks = set(chunks)

        q_c = set(key_ex(query))

        val = sorted(chunks, key = lambda x : x[1], reverse = True )
#         print(val)



        print("\n\nFrom Specification")
        final_val = sel_top(val)
        print("\n\nFrom User Description")
        final_val_query = sel_top(q_c) 
        max_sim = []
#         print(final_val_query)
        for q in final_val_query :

            final_val.append(q)
        #     print(doc)
            print("\n\n Checking distance of {} with the rest {} one by one" .format(final_val[-1], final_val[:]))
#             print(final_val)
            document_embeddings = sbert_model.encode(final_val)

            pairwise_similarities=cosine_similarity(document_embeddings)
            pairwise_differences=euclidean_distances(document_embeddings)
            max_sim.append(most_similar(-1,pairwise_similarities,'Cosine Similarity',final_val))
    #         most_similar(-1,pairwise_differences,'Euclidean Distance', final_val)

            final_val = final_val[:-1]
        max_val = max(max_sim)
        print("\n\n max_sim ",max_val)
        if max_val >= 0.85:
             result.append('yes')
        else:
            result.append("no")
        print("\n\n")
    return result



def extract_quants(data):
    data = data.replace('\n', ' ')
    data = data.replace(';', ' ')
    data = data.replace(',', ' ')
    data = data.replace('’', ' ')
    data = data.replace('‘', ' ')
    data = data.replace('  '," ")
    data = data.replace('   '," ")
    data = data.replace('#'," ")
    data = re.sub(r'\s+'," ",data)
    data = re.sub(r'\s+$',"",data)
    data = re.sub(r'(\(|\[|\{).*(\)|\]|\})',"",data)
    return data

def reduce_data(df1):
    db=pd.DataFrame(columns=["hsn","description"])
    for i in range(len(df1)):
        l= df1.iloc[i,1]
        l= re.sub("(\(|\[|\{).*$","",l)
        l= re.sub("\d{2,}","",l)
        flag=1
        for j in range(len(db)):
            k = db.iloc[j,1]
            if(fuzz.token_set_ratio(l.lower(),k.lower())>90):
                flag=0
        if flag==1:
            db=db.append(pd.DataFrame({"hsn":df1.iloc[i,0],"description":l},index=[0]), ignore_index=True)
    return db

def key_from_description(df):
    fin=pd.DataFrame(columns=["hsn","description"])
    keys = []
    for i in range(len(df)):
        try:
            unit= extract_quants(df.iloc[i]['description'])
        except:
            unit= extract_quants(df.iloc[i]['Description'])
        pattern1 = """
        P1: {<NN|NNP|NNS|DT|CC|VBZ|JJ|IN|VBN>?<NN|NNP|NNS>}
        """
        PChunker = RegexpParser(pattern1)
        res = ""
        try:
            result=PChunker.parse(pos_tag(unit.split()))

            for node in result:
                if node.label()=="P1":
                    tmp=""
                    for leaf in node.leaves():
                        tmp+=leaf[0]+" "
                    print(tmp)
                    res= res + tmp + ", "


    #             else:
    #                 fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,1],"description":"NULL"},index=[0]), ignore_index=True)
        except:
            pass
    #         res.append("NULL")
        print("Res : ", res)
    #         fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,1],"description":"NULL"},index=[0]), ignore_index=True)
        keys.append(res)
    try:
        fin['hsn'] = df['hsn'].values
    except:
        fin['hsn'] = df['HSN_Code'].values

    fin['description'] = keys
    #     fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,1],"description":res},index=[0]), ignore_index=True)
    #         
            #fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,0],"description":""},index=[0]), ignore_index=True)
    # fin= reduce_data(fin)
    print(fin)
    return fin

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel("Test file-1.xlsx")
query = "AC Motor for industrial use"
#result = semantic_check(query, df)

In [4]:
df.head(50)

,S.No,HSN_Code,Description,Unnamed: 3
0,1,8501.10.20,AC motor,Electric motors and generators (excluding gene...
1,4,8501.10.99,DC motors,Electric motors and generators (excluding gene...
2,5,8501.10.10,Fan motor fitted with revolving mechanism,Electric motors and generators (excluding gene...
3,6,8501.10.11,Micro motor,Electric motors and generators (excluding gene...
4,7,8501.10.00,Motors of an output not exceeding 37.5 ...,Electric motors and generators (excluding gene...
5,8,8501.10.00,Motors of an output not exceeding 37.5 W,Electric motors and generators (excluding gene...
6,9,8501.10.90.00,Motors For Other Than Vehicles Of An Output =<...,Electric motors and generators (excluding gene...
7,10,8501.10.10.00,Motors For Vehicles Of An Output =< 37.5W,Electric motors and generators (excluding gene...
8,11,8501.10.00,Motors of an output not,Electric motors and generators (excluding gene...
9,12,8501.10.00,Motors of an output not exceeding 37.5 W,Electric motors and generators (excluding gene...


In [5]:
# Works after preprocessing the text like bracket removal and all

def keep_unique(mystring):

    words = mystring.split()

    answer = [each_pair[0] for each_pair in zip(words, words[1:]) if each_pair[0] != each_pair[1]] + [words[-1]]

    print(' '.join(answer))

In [6]:
import nltk
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
from nltk.stem import WordNetLemmatizer

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [9]:
def stem_lemmatize_text(text):
    text = text.lower()
    l = [stemmer.stem(w) for w in w_tokenizer.tokenize(text)]
    ls =  [lemmatizer.lemmatize(w) for w in l]
    listToStr = ' '.join([str(elem) for elem in ls])
    return listToStr

In [10]:
df['Des_lemmatized'] = df.Description.apply(stem_lemmatize_text)

In [11]:
df['Des_lemmatized'].head()

0                                ac motor
1                                dc motor
2        fan motor fit with revolv mechan
3                             micro motor
4    motor of an output not exceed 37.5 w
Name: Des_lemmatized, dtype: object

In [12]:
res  = []

In [33]:
def jaccard_sim(query):
    res = []
    for i in df['Des_lemmatized']:
        q = stem_lemmatize_text(query)
        li = i.split()
        lq = q.split()
        s1 = set(li)
        s2 = set(lq)
        res.append(float(len(s1.intersection(s2)) / 5))
    return res

In [34]:
jaccard_sim("ac fan motor revolv")

[0.4,
 0.2,
 0.6,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.0,
 0.0,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2]

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
import texthero as hero
df['tfidf'] = hero.do_tfidf(df['Des_lemmatized'])

In [37]:
df['tfidf'][0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9283528855987047,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.37170004008684004,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

# idf lookup list

In [38]:
tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(df['Des_lemmatized'])

idf = tf.idf_ 

In [39]:
print(idf)

[3.19722458 3.19722458 3.19722458 3.19722458 2.09861229 3.19722458
 2.79175947 3.19722458 3.19722458 2.79175947 1.69314718 2.79175947
 2.09861229 3.19722458 3.19722458 2.5040774  3.19722458 3.19722458
 3.19722458 3.19722458 3.19722458 1.11778304 1.94446161 1.58778666
 3.19722458 2.79175947 1.69314718 3.19722458 3.19722458 3.19722458
 3.19722458 3.19722458 3.19722458 3.19722458 3.19722458 2.79175947
 3.19722458 3.19722458]


In [40]:
tf.idf_[tf.vocabulary_["motor"]]

1.1177830356563834

# Create query idf Vector

In [41]:
def create_query_idf_vector(query):
    idf_v = []
    l = query.split()
    for q in l:
        idf_v.append(tf.idf_[tf.vocabulary_[q]])
    return idf_v

In [53]:
q = create_query_idf_vector("fan motor")

In [54]:
print(q)

[3.1972245773362196, 1.1177830356563834]


In [55]:
js = jaccard_sim("fan motor")

In [56]:
df.head()

,S.No,HSN_Code,Description,Unnamed: 3,Des_lemmatized,tfidf
0,1,8501.10.20,AC motor,Electric motors and generators (excluding gene...,ac motor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4,8501.10.99,DC motors,Electric motors and generators (excluding gene...,dc motor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,5,8501.10.10,Fan motor fitted with revolving mechanism,Electric motors and generators (excluding gene...,fan motor fit with revolv mechan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,6,8501.10.11,Micro motor,Electric motors and generators (excluding gene...,micro motor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,7,8501.10.00,Motors of an output not exceeding 37.5 ...,Electric motors and generators (excluding gene...,motor of an output not exceed 37.5 w,"[0.0, 0.0, 0.0, 0.0, 0.44648012504229373, 0.0,..."


In [57]:
print(js)

[0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2]


In [58]:
def prod(qs,j):
    res = []
    for i in j:
        ls = [i*x for x in qs]
        res.append(ls)
    return res

In [59]:
res = prod(q,js)

In [60]:
res[0]

[0.6394449154672439, 0.22355660713127667]

In [61]:

import numpy as np
def mag(res):
    r = []
    for i in res:
        x = np.array(i)
        ans= np.linalg.norm(i)
        r.append(ans)
    return r
        


In [62]:
r = mag(res)

In [63]:
print(r)

[0.6773974878230349, 0.6773974878230349, 1.3547949756460698, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.0, 0.0, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349]
